In [1]:
# константы для sp. z o.o.
spzoo_cit = 0.19 # налог на прибыль юр. лица
spzoo_zus = 0.07
spzoo_tax = 0.19 # налог на дивиденды

In [2]:
# константы jdg
cits = {}

In [3]:
# Skala podatkowa (na zasadach ogólnych, общий налог) - налог доступный по умолчанию при регистрации JDG и единственный доступный для работников по найму (UoP).
# Составляет 12% для доходов от 30 до 120 тыс. зл. и 32% - для всего, что выше.
# Доход до 30 тыс. зл. налогом не облагается.
# Этот налог позволяет уменьшить налогооблагаемый доход за счет необходимых для деятельности расходов (ноутбук, монитор, бухгалтерия, ПО и т.д.).
# Также позволяет подать налоговую декларацию вместе с супругом, что при отсутствии дохода у одного из супругов, уменьшает налогооблагаемый доход за счет дополнительных 30 тыс. зл. не облагаемого дохода.
cits["jdg.og"] = (
    (30000, 0), # от 0 до 30к pln
    (120000, 0.12), # от 30 до 120к pln
    (1000000000, 0.32), # от 120к pln
)

In [4]:
# Podatek liniowy (пропорциональный налог) - некогда (до популяризации ryczałtu) самый выгодный налог в едином размере 19% вне зависимости от размера доходов для тех,
# значительная часть дохода которых попадала под 32% общего налога.
# Отличительной особенностью пропорционального налога (podatek liniowy) на фоне фиксированного (ryczałt) является возможность уменьшения налогооблагаемой суммы за счет расходов, так же как и в случае с общим налогом (skala podatkowa).
# Но, в отличии от общего налога, доход до 30 тыс. зл. налогом облагается, а совместная подача налоговой декларации с супругом недоступна.
jdg_lin_cit = (
   (1000000000, 0.19), # от 0
)

In [5]:
# Ryczałt (фиксированный налог) - ныне очень популярный налог в размере от 2% до 17% в зависимости от вида деятельности JDG.
# Для программистов - 12%.
# Важный нюанс этого налога в том, что расходы никак не влияют на налогооблагаемый доход, т.е. налог рассчитывается исключительно на основании полученного дохода.
# Так как и при пропорциональном налоге (podatek liniowy), доход до 30 тыс. зл. налогом облагается, а подача налоговой декларации с супругом недоступна.
jdg_rycz_cit = (
   (1000000000, 0.12), # от 0
)
jdg_rycz_cit_cons = 0.32

In [6]:
# выручка - revenue
income = 25000 # доход - income
# прибыль - profit
expense = 0 # расход - expense

In [7]:
# sp. z o.o.
# 1 фаундер
print((income - expense) * spzoo_cit)

4750.0


In [27]:
# jdg - na zasadach ogólnych
monthly_profits = [0] * 12
cit_map = cits["jdg.og"]
cit_idx = 0
print("income={} expense={} cits={}".format(income, expense, cit_map))

month, month_profit, month_income, month_expense = -1, 0, income, expense
while True:
    month+=1
    if month > 11:
        break
    cit_base = month_income - month_expense
    cit_check = sum(monthly_profits) + cit_base
    cit_limit, cit_value = cit_map[cit_idx]
    print("[{}] base={} total={} {} taxrange={} ".format(month+1, cit_base, sum(monthly_profits), monthly_profits, cit_map[cit_idx]))
    if cit_limit - cit_check >= 0:
        # накопленная прибыль внутри рамок данного налогового диапазона - на всю прибыль один налог
        monthly_profits[month] += cit_base * (1 - cit_value)
        month_profit, month_income, month_expense = 0, income, expense
    else:
        # накопленная прибыль превышает лимит
        # на часть внутри - данные лимиты
        low = cit_check - cit_limit
        monthly_profits[month] += low * (1 - cit_value)

        high = cit_base - low
        print("[{}] !!! low={} high={}".format(month+1, low, high))
        # превышение -> переносим на "фейковый месяц"
        month_income, month_expense = high, 0
        cit_idx += 1 # следующий налоговый диапазон
        month-=1
    #print("[{}] sum={} {} tax={}".format(month, sum(monthly_profits), monthly_profits, 0))
print("TOTAL: {} {}".format(sum(monthly_profits), monthly_profits))


income=25000 expense=0 cits=((30000, 0), (120000, 0.12), (1000000000, 0.32))
[1] base=25000 total=0 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] taxrange=(30000, 0) 
[2] base=25000 total=25000 [25000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] taxrange=(30000, 0) 
[2] !!! low=20000 high=5000
[2] base=5000 total=45000 [25000, 20000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] taxrange=(120000, 0.12) 
[3] base=25000 total=49400.0 [25000, 24400.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] taxrange=(120000, 0.12) 
[4] base=25000 total=71400.0 [25000, 24400.0, 22000.0, 0, 0, 0, 0, 0, 0, 0, 0, 0] taxrange=(120000, 0.12) 
[5] base=25000 total=93400.0 [25000, 24400.0, 22000.0, 22000.0, 0, 0, 0, 0, 0, 0, 0, 0] taxrange=(120000, 0.12) 
[6] base=25000 total=115400.0 [25000, 24400.0, 22000.0, 22000.0, 22000.0, 0, 0, 0, 0, 0, 0, 0] taxrange=(120000, 0.12) 
[6] !!! low=20400.0 high=4600.0
[6] base=4600.0 total=133352.0 [25000, 24400.0, 22000.0, 22000.0, 22000.0, 17952.0, 0, 0, 0, 0, 0, 0] taxrange=(1000000000, 0.32) 
[7] base=25000 total=1364

In [ ]:
# jdg - liniowy

In [ ]:
# jdg - ryczałt